# Importing training Data
TODO: dealing with test data loading

In [1]:
import numpy as np
from glob import glob
import omegaconf
from utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd


In [4]:
import os
os.makedirs('test', exist_ok=True)
np.save('test/1.npy', np.array([1, 2, 3]))

In [2]:
conf = omegaconf.OmegaConf.load('configs/baseConfig.yaml')

In [13]:
rootDIR = '/home/bdudas/PCT_DATA2/output/'
wpts = list(range(70,200,10))
sampleWPT = wpts[0]
sampleElem = glob(rootDIR + f'wpt_{sampleWPT}/*.npy')[0]
sampleElem

'/home/bdudas/PCT_DATA2/output/wpt_70/1740420303_2e8d5efa-1b4a-4125-9745-e3217cefe176_water_1000Primaries_2.hits.npy'

In [17]:
hit = pd.DataFrame(np.load(sampleElem))
hit = hit[hit.parentID == 0]

In [23]:
hit['Layer'] =  2*(hit['volumeID[2]'])+hit['volumeID[3]']

In [25]:
hit.sort_values(by=['eventID','Layer'],inplace=True)

In [37]:
# Group by eventID and check for non-unique Layer values
non_unique_layers = hit.groupby('eventID')['Layer'].nunique() != hit.groupby('eventID')['Layer'].size()

# Filter the eventIDs with non-unique Layer values
non_unique_eventIDs = non_unique_layers[non_unique_layers].index

# Display the eventIDs with non-unique Layer values
print("EventIDs with non-unique Layer values:", non_unique_eventIDs.tolist())
hit = hit[~hit.eventID.isin(non_unique_eventIDs)]

EventIDs with non-unique Layer values: [45, 266, 718, 799]


In [59]:

tmp = hit.loc[hit.eventID == 0,['eventID','posX','posY','edep']]
tmp.head(3)

,eventID,posX,posY,edep
0,0,-12.803946,1.457997,0.015062
1,0,-14.391742,1.822247,0.018286
2,0,-15.460902,2.131060,0.029171


In [61]:
tmp.drop(columns='eventID',inplace=True)

In [74]:
def padarray(A,size = 41):
    t = size - len(A)
    if t > 0:
        return np.pad(A, [(0,t),(0,0)], 'constant', constant_values=0)
    else:
        return A

In [75]:
padtmp = padarray(tmp.values)
padtmp.shape

(41, 3)

In [76]:
padtmp

array([[-1.2803946e+01,  1.4579966e+00,  1.5062129e-02],
       [-1.4391742e+01,  1.8222469e+00,  1.8286265e-02],
       [-1.5460902e+01,  2.1310601e+00,  2.9171370e-02],
       [-1.5590895e+01,  2.1430550e+00,  1.4753000e-02],
       [-1.5722945e+01,  2.1501043e+00,  1.9859070e-02],
       [-1.5923079e+01,  2.1365724e+00,  1.7760776e-02],
       [-1.6221889e+01,  2.1059155e+00,  1.1343314e-02],
       [-1.6494389e+01,  2.0766759e+00,  1.1815625e-02],
       [-1.6728987e+01,  2.0504918e+00,  2.1682428e-02],
       [-1.6936878e+01,  2.0021400e+00,  1.9102644e-02],
       [-1.7062344e+01,  1.9008839e+00,  1.3040190e-02],
       [-1.7192095e+01,  1.7882410e+00,  2.3866680e-02],
       [-1.7335812e+01,  1.6563423e+00,  2.7240906e-02],
       [-1.7463354e+01,  1.5715796e+00,  2.5114294e-02],
       [-1.7519327e+01,  1.5409077e+00,  2.0305952e-02],
       [-1.7617891e+01,  1.4087247e+00,  3.9654348e-02],
       [-1.7697517e+01,  1.2714899e+00,  1.4863799e-02],
       [-1.7809317e+01,  1.0998

# Model testings

In [6]:
from utils.model import PosPredictor

/home/bdudas/anaconda3/envs/pct/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

/home/bdudas/anaconda3/envs/pct/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
